In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!nvidia-smi

Sat Sep 12 04:59:16 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
! mkdir /root/.kaggle
! cp '/content/drive/My Drive/kaggle.json' /root/.kaggle
! chmod 400 /root/.kaggle/kaggle.json

!pip uninstall -y kaggle >> quit
!pip install --upgrade pip >> quit
!pip install kaggle==1.5.6 >> quit
!kaggle -v >> quit

In [ ]:
%%time
!kaggle datasets download -d gopidurgaprasad/birdsong-stage1-1sec
!unzip birdsong-stage1-1sec.zip >> quit
!rm birdsong-stage1-1sec.zip

100% 8.67G/8.68G [03:40<00:00, 59.1MB/s]
100% 8.68G/8.68G [03:40<00:00, 42.2MB/s]
CPU times: user 1.41 s, sys: 282 ms, total: 1.69 s
Wall time: 9min 7s


In [ ]:
%%time
!kaggle datasets download -d gopidurgaprasad/birdsong-stage1-1sec-sudo-5
!unzip birdsong-stage1-1sec-sudo-5.zip >> quit
!rm birdsong-stage1-1sec-sudo-5.zip

100% 3.26G/3.26G [01:49<00:00, 61.6MB/s]
100% 3.26G/3.26G [01:49<00:00, 31.9MB/s]
CPU times: user 501 ms, sys: 98.4 ms, total: 599 ms
Wall time: 3min 23s


In [ ]:
!pip install soundfile >> quit
!pip install catalyst >> quit
!pip install torchaudio >> quit
!pip install torchlibrosa >> quit
!pip install timm >> quit
!pip install audiomentations >> quit

In [ ]:
import glob
import numpy as np
import pandas as pd

from sklearn.model_selection import StratifiedKFold

In [ ]:
stage1_wav = glob.glob("/content/train_stage1_1sec/tmp/birdsongWav/train_stage1_1sec/*.wav")
stage1_sudo_wav = glob.glob("/content/train_stage1_1sec_sudo/tmp/birdsongWav/train_stage1_1sec_sudo/*.wav")

stage1_wav = stage1_wav + stage1_sudo_wav

stage1_wav_df = pd.DataFrame()
stage1_wav_df["path"] = stage1_wav
stage1_wav_df["filename"] = stage1_wav_df["path"].apply(lambda x: x.split('/')[-1].strip('.wav'))

stage1_df = pd.read_csv("stage1_df.csv")
stage1_df["filename"] = stage1_df["file_name"].apply(lambda x: x.split('/')[-1].strip('.wav'))
stage1_df = stage1_df.merge(stage1_wav_df, on="filename", how="left")

stage1_sudo_df = pd.read_csv("stage1_sudo.csv")
stage1_sudo_df["filename"] = stage1_sudo_df["file_name"].apply(lambda x: x.split('/')[-1].strip('.wav'))
stage1_sudo_df = stage1_sudo_df.merge(stage1_wav_df, on="filename", how="left")

stage1_df = stage1_df.append(stage1_sudo_df)

stage1_df.head()

,ebird_code,file_name,pred_label,pred_prob,pred_code,kfold,filename,path
0,logshr,../input/birdsong-wav-02/tmp/birdsongWav/train...,146,1.0,logshr,1,XC255302_31,/content/train_stage1_1sec/tmp/birdsongWav/tra...
1,eastow,../input/birdsong-wav-01/tmp/birdsongWav/train...,93,1.0,eastow,3,XC319606_46,/content/train_stage1_1sec/tmp/birdsongWav/tra...
2,easblu,../input/birdsong-wav-01/tmp/birdsongWav/train...,89,1.0,easblu,0,XC502284_26,/content/train_stage1_1sec/tmp/birdsongWav/tra...
3,linspa,../input/birdsong-wav-02/tmp/birdsongWav/train...,143,1.0,linspa,1,XC483715_35,/content/train_stage1_1sec/tmp/birdsongWav/tra...
4,linspa,../input/birdsong-wav-02/tmp/birdsongWav/train...,143,1.0,linspa,1,XC445186_62,/content/train_stage1_1sec/tmp/birdsongWav/tra...


In [ ]:
stage1_df.shape

(508712, 8)

In [ ]:
stage1_df.kfold.value_counts()

0    106505
2    102991
1    102384
3    101895
4     94937
Name: kfold, dtype: int64

In [ ]:
stage1_df.to_csv("drive/My Drive/Cornell Birdcall Identification/input/stage1_sudo.csv", index=False)

In [ ]:
%%writefile pytorch_utils.py

import numpy as np
import time
import torch
import torch.nn as nn


def move_data_to_device(x, device):
    if 'float' in str(x.dtype):
        x = torch.Tensor(x)
    elif 'int' in str(x.dtype):
        x = torch.LongTensor(x)
    else:
        return x

    return x.to(device)


def do_mixup(x, mixup_lambda):
    """Mixup x of even indexes (0, 2, 4, ...) with x of odd indexes 
    (1, 3, 5, ...).
    Args:
      x: (batch_size * 2, ...)
      mixup_lambda: (batch_size * 2,)
    Returns:
      out: (batch_size, ...)
    """
    
    out = (x[0 :: 2].transpose(0, -1) * mixup_lambda[0 :: 2] + \
        x[1 :: 2].transpose(0, -1) * mixup_lambda[1 :: 2]).transpose(0, -1)
        
    return out

class Mixup(object):
    def __init__(self, mixup_alpha, random_seed=1234):
        """Mixup coefficient generator.
        """
        self.mixup_alpha = mixup_alpha
        self.random_state = np.random.RandomState(random_seed)

    def get_lambda(self, batch_size):
        """Get mixup random coefficients.
        Args:
          batch_size: int
        Returns:
          mixup_lambdas: (batch_size,)
        """
        mixup_lambdas = []
        for n in range(0, batch_size, 2):
            lam = self.random_state.beta(self.mixup_alpha, self.mixup_alpha, 1)[0]
            mixup_lambdas.append(lam)
            mixup_lambdas.append(1. - lam)

        return np.array(mixup_lambdas)
    

def append_to_dict(dict, key, value):
    if key in dict.keys():
        dict[key].append(value)
    else:
        dict[key] = [value]


def forward(model, generator, return_input=False, 
    return_target=False):
    """Forward data to a model.
    
    Args: 
      model: object
      generator: object
      return_input: bool
      return_target: bool
    Returns:
      audio_name: (audios_num,)
      clipwise_output: (audios_num, classes_num)
      (ifexist) segmentwise_output: (audios_num, segments_num, classes_num)
      (ifexist) framewise_output: (audios_num, frames_num, classes_num)
      (optional) return_input: (audios_num, segment_samples)
      (optional) return_target: (audios_num, classes_num)
    """
    output_dict = {}
    device = next(model.parameters()).device
    time1 = time.time()

    # Forward data to a model in mini-batches
    for n, batch_data_dict in enumerate(generator):
        print(n)
        batch_waveform = move_data_to_device(batch_data_dict['waveform'], device)
        
        with torch.no_grad():
            model.eval()
            batch_output = model(batch_waveform)

        append_to_dict(output_dict, 'audio_name', batch_data_dict['audio_name'])

        append_to_dict(output_dict, 'clipwise_output', 
            batch_output['clipwise_output'].data.cpu().numpy())

        if 'segmentwise_output' in batch_output.keys():
            append_to_dict(output_dict, 'segmentwise_output', 
                batch_output['segmentwise_output'].data.cpu().numpy())

        if 'framewise_output' in batch_output.keys():
            append_to_dict(output_dict, 'framewise_output', 
                batch_output['framewise_output'].data.cpu().numpy())
            
        if return_input:
            append_to_dict(output_dict, 'waveform', batch_data_dict['waveform'])
            
        if return_target:
            if 'target' in batch_data_dict.keys():
                append_to_dict(output_dict, 'target', batch_data_dict['target'])

        if n % 10 == 0:
            print(' --- Inference time: {:.3f} s / 10 iterations ---'.format(
                time.time() - time1))
            time1 = time.time()

    for key in output_dict.keys():
        output_dict[key] = np.concatenate(output_dict[key], axis=0)

    return output_dict


def interpolate(x, ratio):
    """Interpolate data in time domain. This is used to compensate the 
    resolution reduction in downsampling of a CNN.
    
    Args:
      x: (batch_size, time_steps, classes_num)
      ratio: int, ratio to interpolate
    Returns:
      upsampled: (batch_size, time_steps * ratio, classes_num)
    """
    (batch_size, time_steps, classes_num) = x.shape
    upsampled = x[:, :, None, :].repeat(1, 1, ratio, 1)
    upsampled = upsampled.reshape(batch_size, time_steps * ratio, classes_num)
    return upsampled


def pad_framewise_output(framewise_output, frames_num):
    """Pad framewise_output to the same length as input frames. The pad value 
    is the same as the value of the last frame.
    Args:
      framewise_output: (batch_size, frames_num, classes_num)
      frames_num: int, number of frames to pad
    Outputs:
      output: (batch_size, frames_num, classes_num)
    """
    pad = framewise_output[:, -1 :, :].repeat(1, frames_num - framewise_output.shape[1], 1)
    """tensor for padding"""

    output = torch.cat((framewise_output, pad), dim=1)
    """(batch_size, frames_num, classes_num)"""

    return output


def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


def count_flops(model, audio_length):
    """Count flops. Code modified from others' implementation.
    """
    multiply_adds = True
    list_conv2d=[]
    def conv2d_hook(self, input, output):
        batch_size, input_channels, input_height, input_width = input[0].size()
        output_channels, output_height, output_width = output[0].size()
 
        kernel_ops = self.kernel_size[0] * self.kernel_size[1] * (self.in_channels / self.groups) * (2 if multiply_adds else 1)
        bias_ops = 1 if self.bias is not None else 0
 
        params = output_channels * (kernel_ops + bias_ops)
        flops = batch_size * params * output_height * output_width
 
        list_conv2d.append(flops)

    list_conv1d=[]
    def conv1d_hook(self, input, output):
        batch_size, input_channels, input_length = input[0].size()
        output_channels, output_length = output[0].size()
 
        kernel_ops = self.kernel_size[0] * (self.in_channels / self.groups) * (2 if multiply_adds else 1)
        bias_ops = 1 if self.bias is not None else 0
 
        params = output_channels * (kernel_ops + bias_ops)
        flops = batch_size * params * output_length
 
        list_conv1d.append(flops)
 
    list_linear=[] 
    def linear_hook(self, input, output):
        batch_size = input[0].size(0) if input[0].dim() == 2 else 1
 
        weight_ops = self.weight.nelement() * (2 if multiply_adds else 1)
        bias_ops = self.bias.nelement()
 
        flops = batch_size * (weight_ops + bias_ops)
        list_linear.append(flops)
 
    list_bn=[] 
    def bn_hook(self, input, output):
        list_bn.append(input[0].nelement() * 2)
 
    list_relu=[] 
    def relu_hook(self, input, output):
        list_relu.append(input[0].nelement() * 2)
 
    list_pooling2d=[]
    def pooling2d_hook(self, input, output):
        batch_size, input_channels, input_height, input_width = input[0].size()
        output_channels, output_height, output_width = output[0].size()
 
        kernel_ops = self.kernel_size * self.kernel_size
        bias_ops = 0
        params = output_channels * (kernel_ops + bias_ops)
        flops = batch_size * params * output_height * output_width
 
        list_pooling2d.append(flops)

    list_pooling1d=[]
    def pooling1d_hook(self, input, output):
        batch_size, input_channels, input_length = input[0].size()
        output_channels, output_length = output[0].size()
 
        kernel_ops = self.kernel_size[0]
        bias_ops = 0
        
        params = output_channels * (kernel_ops + bias_ops)
        flops = batch_size * params * output_length
 
        list_pooling2d.append(flops)
 
    def foo(net):
        childrens = list(net.children())
        if not childrens:
            if isinstance(net, nn.Conv2d):
                net.register_forward_hook(conv2d_hook)
            elif isinstance(net, nn.Conv1d):
                net.register_forward_hook(conv1d_hook)
            elif isinstance(net, nn.Linear):
                net.register_forward_hook(linear_hook)
            elif isinstance(net, nn.BatchNorm2d) or isinstance(net, nn.BatchNorm1d):
                net.register_forward_hook(bn_hook)
            elif isinstance(net, nn.ReLU):
                net.register_forward_hook(relu_hook)
            elif isinstance(net, nn.AvgPool2d) or isinstance(net, nn.MaxPool2d):
                net.register_forward_hook(pooling2d_hook)
            elif isinstance(net, nn.AvgPool1d) or isinstance(net, nn.MaxPool1d):
                net.register_forward_hook(pooling1d_hook)
            else:
                print('Warning: flop of module {} is not counted!'.format(net))
            return
        for c in childrens:
            foo(c)

    # Register hook
    foo(model)
    
    device = device = next(model.parameters()).device
    input = torch.rand(1, audio_length).to(device)

    out = model(input)
 
    total_flops = sum(list_conv2d) + sum(list_conv1d) + sum(list_linear) + \
        sum(list_bn) + sum(list_relu) + sum(list_pooling2d) + sum(list_pooling1d)
    
    return total_flops

Writing pytorch_utils.py


In [ ]:
%%writefile classifiers.py

import numpy as np
from functools import partial

import torch
import torch.nn as nn
import torch.nn.functional as F
import timm
from timm.models.efficientnet import tf_efficientnet_b4_ns, tf_efficientnet_b3_ns, \
    tf_efficientnet_b5_ns, tf_efficientnet_b2_ns, tf_efficientnet_b6_ns, tf_efficientnet_b7_ns, tf_efficientnet_b0_ns
from torch import nn
from torch.nn.modules.dropout import Dropout
from torch.nn.modules.linear import Linear
from torch.nn.modules.pooling import AdaptiveAvgPool2d, AdaptiveMaxPool2d

from torchlibrosa.stft import Spectrogram, LogmelFilterBank
from torchlibrosa.augmentation import SpecAugmentation

from pytorch_utils import do_mixup, interpolate, pad_framewise_output

encoder_params = {
    "resnest50d" : {
        "features" : 2048,
        "init_op"  : partial(timm.models.resnest50d, pretrained=True, in_chans=1)
    },
    "densenet201" : {
        "features": 1920,
        "init_op": partial(timm.models.densenet201, pretrained=True)
    },
    "dpn92" : {
        "features": 2688,
        "init_op": partial(timm.models.dpn92, pretrained=True)
    },
    "dpn131": {
        "features": 2688,
        "init_op": partial(timm.models.dpn131, pretrained=True)
    },
    "tf_efficientnet_b0_ns": {
        "features": 1280,
        "init_op": partial(tf_efficientnet_b0_ns, pretrained=True, drop_path_rate=0.2)
    },
    "tf_efficientnet_b3_ns": {
        "features": 1536,
        "init_op": partial(tf_efficientnet_b3_ns, pretrained=True, drop_path_rate=0.2)
    },
    "tf_efficientnet_b2_ns": {
        "features": 1408,
        "init_op": partial(tf_efficientnet_b2_ns, pretrained=True, drop_path_rate=0.2)
    }
}


def init_layer(layer):
    """Initialize a Linear or Convolutional layer. """
    nn.init.xavier_uniform_(layer.weight)
 
    if hasattr(layer, 'bias'):
        if layer.bias is not None:
            layer.bias.data.fill_(0.)
            
    
def init_bn(bn):
    """Initialize a Batchnorm layer. """
    bn.bias.data.fill_(0.)
    bn.weight.data.fill_(1.)


class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        
        super(ConvBlock, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=in_channels, 
                              out_channels=out_channels,
                              kernel_size=(3, 3), stride=(1, 1),
                              padding=(1, 1), bias=False)
                              
        self.conv2 = nn.Conv2d(in_channels=out_channels, 
                              out_channels=out_channels,
                              kernel_size=(3, 3), stride=(1, 1),
                              padding=(1, 1), bias=False)
                              
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.init_weight()
        
    def init_weight(self):
        init_layer(self.conv1)
        init_layer(self.conv2)
        init_bn(self.bn1)
        init_bn(self.bn2)

        
    def forward(self, input, pool_size=(2, 2), pool_type='avg'):
        
        x = input
        x = F.relu_(self.bn1(self.conv1(x)))
        x = F.relu_(self.bn2(self.conv2(x)))
        if pool_type == 'max':
            x = F.max_pool2d(x, kernel_size=pool_size)
        elif pool_type == 'avg':
            x = F.avg_pool2d(x, kernel_size=pool_size)
        elif pool_type == 'avg+max':
            x1 = F.avg_pool2d(x, kernel_size=pool_size)
            x2 = F.max_pool2d(x, kernel_size=pool_size)
            x = x1 + x2
        else:
            raise Exception('Incorrect argument!')
        
        return x


class ConvBlock5x5(nn.Module):
    def __init__(self, in_channels, out_channels):
        
        super(ConvBlock5x5, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=in_channels, 
                              out_channels=out_channels,
                              kernel_size=(5, 5), stride=(1, 1),
                              padding=(2, 2), bias=False)
                              
        self.bn1 = nn.BatchNorm2d(out_channels)

        self.init_weight()
        
    def init_weight(self):
        init_layer(self.conv1)
        init_bn(self.bn1)

        
    def forward(self, input, pool_size=(2, 2), pool_type='avg'):
        
        x = input
        x = F.relu_(self.bn1(self.conv1(x)))
        if pool_type == 'max':
            x = F.max_pool2d(x, kernel_size=pool_size)
        elif pool_type == 'avg':
            x = F.avg_pool2d(x, kernel_size=pool_size)
        elif pool_type == 'avg+max':
            x1 = F.avg_pool2d(x, kernel_size=pool_size)
            x2 = F.max_pool2d(x, kernel_size=pool_size)
            x = x1 + x2
        else:
            raise Exception('Incorrect argument!')
        
        return x


class AttBlock(nn.Module):
    def __init__(self, n_in, n_out, activation='linear', temperature=1.):
        super(AttBlock, self).__init__()
        
        self.activation = activation
        self.temperature = temperature
        self.att = nn.Conv1d(in_channels=n_in, out_channels=n_out, kernel_size=1, stride=1, padding=0, bias=True)
        self.cla = nn.Conv1d(in_channels=n_in, out_channels=n_out, kernel_size=1, stride=1, padding=0, bias=True)
        
        self.bn_att = nn.BatchNorm1d(n_out)
        self.init_weights()
        
    def init_weights(self):
        init_layer(self.att)
        init_layer(self.cla)
        init_bn(self.bn_att)
         
    def forward(self, x):
        # x: (n_samples, n_in, n_time)
        norm_att = torch.softmax(torch.clamp(self.att(x), -10, 10), dim=-1)
        cla = self.nonlinear_transform(self.cla(x))
        x = torch.sum(norm_att * cla, dim=2)
        return x, norm_att, cla

    def nonlinear_transform(self, x):
        if self.activation == 'linear':
            return x
        elif self.activation == 'sigmoid':
            return torch.sigmoid(x)


class Cnn14_16k(nn.Module):
    def __init__(self, sample_rate, window_size, hop_size, mel_bins, fmin, fmax, classes_num):
        
        super(Cnn14_16k, self).__init__() 

        assert sample_rate == 16000
        assert window_size == 512
        assert hop_size == 160
        assert mel_bins == 64
        assert fmin == 50
        assert fmax == 8000

        window = 'hann'
        center = True
        pad_mode = 'reflect'
        ref = 1.0
        amin = 1e-10
        top_db = None

        # Spectrogram extractor
        self.spectrogram_extractor = Spectrogram(n_fft=window_size, hop_length=hop_size, 
            win_length=window_size, window=window, center=center, pad_mode=pad_mode, 
            freeze_parameters=True)

        # Logmel feature extractor
        self.logmel_extractor = LogmelFilterBank(sr=sample_rate, n_fft=window_size, 
            n_mels=mel_bins, fmin=fmin, fmax=fmax, ref=ref, amin=amin, top_db=top_db, 
            freeze_parameters=True)

        # Spec augmenter
        self.spec_augmenter = SpecAugmentation(time_drop_width=64, time_stripes_num=2, 
            freq_drop_width=8, freq_stripes_num=2)

        self.bn0 = nn.BatchNorm2d(64)

        self.conv_block1 = ConvBlock(in_channels=1, out_channels=64)
        self.conv_block2 = ConvBlock(in_channels=64, out_channels=128)
        self.conv_block3 = ConvBlock(in_channels=128, out_channels=256)
        self.conv_block4 = ConvBlock(in_channels=256, out_channels=512)
        self.conv_block5 = ConvBlock(in_channels=512, out_channels=1024)
        self.conv_block6 = ConvBlock(in_channels=1024, out_channels=2048)

        self.fc1 = nn.Linear(2048, 2048, bias=True)
        self.fc_audioset1 = nn.Linear(2048, classes_num, bias=True)
        
        self.init_weight()

    def init_weight(self):
        init_bn(self.bn0)
        init_layer(self.fc1)
        init_layer(self.fc_audioset1)
 
    def forward(self, input, spec_aug=False, mixup_lambda=None):
        """
        Input: (batch_size, data_length)"""

        x = self.spectrogram_extractor(input)   # (batch_size, 1, time_steps, freq_bins)
        x = self.logmel_extractor(x)    # (batch_size, 1, time_steps, mel_bins)
        #x = input
        x = x.transpose(1, 3)
        x = self.bn0(x)
        x = x.transpose(1, 3)

        if spec_aug:
            x = self.spec_augmenter(x)

        # Mixup on spectrogram
        if mixup_lambda is not None:
            x = do_mixup(x, mixup_lambda)
        
        x = self.conv_block1(x, pool_size=(2, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv_block2(x, pool_size=(2, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv_block3(x, pool_size=(2, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv_block4(x, pool_size=(2, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv_block5(x, pool_size=(2, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv_block6(x, pool_size=(1, 1), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = torch.mean(x, dim=3)
        
        (x1, _) = torch.max(x, dim=2)
        x2 = torch.mean(x, dim=2)
        x = x1 + x2
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu_(self.fc1(x))
        #embedding = F.dropout(x, p=0.5, training=self.training)
        clipwise_output = self.fc_audioset1(x)
        
        #output_dict = {'clipwise_output': clipwise_output, 'embedding': embedding}

        return clipwise_output

class BirdClassifier(nn.Module):
    def __init__(self, encoder, sample_rate, window_size, hop_size, mel_bins, fmin, fmax, classes_num):
        super().__init__()

        window = 'hann'
        center = True
        pad_mode = 'reflect'
        ref = 1.0
        amin = 1e-10
        top_db = None
        
        # Spectrogram extractor
        self.spectrogram_extractor = Spectrogram(n_fft=window_size, hop_length=hop_size, 
            win_length=window_size, window=window, center=center, pad_mode=pad_mode, 
            freeze_parameters=True)

        # Logmel feature extractor
        self.logmel_extractor = LogmelFilterBank(sr=sample_rate, n_fft=window_size, 
            n_mels=mel_bins, fmin=fmin, fmax=fmax, ref=ref, amin=amin, top_db=top_db, 
            freeze_parameters=True)

        # Spec augmenter
        self.spec_augmenter = SpecAugmentation(time_drop_width=64, time_stripes_num=2, 
            freq_drop_width=8, freq_stripes_num=2)

        self.encoder = encoder_params[encoder]["init_op"]()
        self.avg_pool = AdaptiveAvgPool2d((1, 1))
        self.dropout = Dropout(0.2)
        self.fc = Linear(encoder_params[encoder]['features'], classes_num)

    def forward(self, input, spec_aug=False, mixup_lambda=None):

        x = self.spectrogram_extractor(input)   # (batch_size, 1, time_steps, freq_bins)
        x = self.logmel_extractor(x)    # (batch_size, 1, time_steps, mel_bins)

        if spec_aug:
            x = self.spec_augmenter(x)

        # Mixup on spectrogram
        if mixup_lambda is not None:
            x = do_mixup(x, mixup_lambda)

        x = self.encoder.forward_features(x)
        x = self.avg_pool(x).flatten(1)
        x = self.dropout(x)
        x = self.fc(x)
        return x


class Cnn14_DecisionLevelAtt(nn.Module):
    def __init__(self, sample_rate, window_size, hop_size, mel_bins, fmin, 
        fmax, classes_num):
        
        super(Cnn14_DecisionLevelAtt, self).__init__()

        window = 'hann'
        center = True
        pad_mode = 'reflect'
        ref = 1.0
        amin = 1e-10
        top_db = None
        self.interpolate_ratio = 32     # Downsampled ratio

        # Spectrogram extractor
        self.spectrogram_extractor2 = Spectrogram(n_fft=window_size, hop_length=hop_size, 
            win_length=window_size, window=window, center=center, pad_mode=pad_mode, 
            freeze_parameters=True)

        # Logmel feature extractor
        self.logmel_extractor2 = LogmelFilterBank(sr=sample_rate, n_fft=window_size, 
            n_mels=mel_bins, fmin=fmin, fmax=fmax, ref=ref, amin=amin, top_db=top_db, 
            freeze_parameters=True)

        # Spec augmenter
        self.spec_augmenter = SpecAugmentation(time_drop_width=64, time_stripes_num=2, 
            freq_drop_width=8, freq_stripes_num=2)

        self.bn0 = nn.BatchNorm2d(64)

        self.conv_block1 = ConvBlock(in_channels=1, out_channels=64)
        self.conv_block2 = ConvBlock(in_channels=64, out_channels=128)
        self.conv_block3 = ConvBlock(in_channels=128, out_channels=256)
        self.conv_block4 = ConvBlock(in_channels=256, out_channels=512)
        self.conv_block5 = ConvBlock(in_channels=512, out_channels=1024)
        self.conv_block6 = ConvBlock(in_channels=1024, out_channels=2048)

        self.fc1 = nn.Linear(2048, 2048, bias=True)
        self.att_block2 = AttBlock(2048, classes_num, activation='sigmoid')
        
        self.init_weight()

    def init_weight(self):
        init_bn(self.bn0)
        init_layer(self.fc1)
 
    def forward(self, input, spec_aug=False, mixup_lambda=None):
        """
        Input: (batch_size, data_length)"""

        x = self.spectrogram_extractor2(input)   # (batch_size, 1, time_steps, freq_bins)
        x = self.logmel_extractor2(x)    # (batch_size, 1, time_steps, mel_bins)

        frames_num = x.shape[2]
        
        x = x.transpose(1, 3)
        x = self.bn0(x)
        x = x.transpose(1, 3)
        
        if spec_aug:
            x = self.spec_augmenter(x)

        # Mixup on spectrogram
        if mixup_lambda is not None:
            x = do_mixup(x, mixup_lambda)
        
        x = self.conv_block1(x, pool_size=(2, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv_block2(x, pool_size=(2, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv_block3(x, pool_size=(2, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv_block4(x, pool_size=(2, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv_block5(x, pool_size=(2, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv_block6(x, pool_size=(1, 1), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = torch.mean(x, dim=3)
        
        x1 = F.max_pool1d(x, kernel_size=3, stride=1, padding=1)
        x2 = F.avg_pool1d(x, kernel_size=3, stride=1, padding=1)
        x = x1 + x2
        x = F.dropout(x, p=0.5, training=self.training)
        x = x.transpose(1, 2)
        x = F.relu_(self.fc1(x))
        x = x.transpose(1, 2)
        x = F.dropout(x, p=0.5, training=self.training)
        (clipwise_output, _, segmentwise_output) = self.att_block2(x)
        segmentwise_output = segmentwise_output.transpose(1, 2)

        # Get framewise output
        framewise_output = interpolate(segmentwise_output, self.interpolate_ratio)
        framewise_output = pad_framewise_output(framewise_output, frames_num)

        output_dict = {'framewise_output': framewise_output, 
            'clipwise_output': clipwise_output}

        return output_dict["clipwise_output"]

Overwriting classifiers.py


In [ ]:
%%writefile dataset.py

import numpy as np
import pandas as pd
import soundfile as sf

from torch.utils.data import Dataset
from typing import Optional, List

import torch
from albumentations.pytorch.functional import img_to_tensor

from bird_codes import BIRD_CODE, INV_BIRD_CODE

class PANNsDataset(Dataset):
    def __init__(
        self,
        file_list: List[List[str]],
        period=1,
        transforms=None,
        train=True,
        waveform_transforms=None):

        self.file_list = file_list
        self.period = period
        self.transforms = transforms
        self.train = train
        self.waveform_transforms = waveform_transforms

    def __len__(self):
        return len(self.file_list)
    
    def __getitem__(self, idx: int):
        
        wav_path, ebird_code, pred_prob = self.file_list[idx]

        y, sr = sf.read(wav_path)

        if not self.train:
            self.period = self.period

        if self.waveform_transforms:
            y = self.waveform_transforms(y, sample_rate=sr)
        else:
            len_y = len(y)
            effective_length = sr * self.period
            if len_y < effective_length:
                new_y = np.zeros(effective_length, dtype=y.dtype)
                start = np.random.randint(effective_length - len_y)
                new_y[start:start + len_y] = y
                y = new_y.astype(np.float32)
            elif len_y > effective_length:
                start = np.random.randint(len_y - effective_length)
                y = y[start:start + effective_length].astype(np.float32)
            else:
                y = y.astype(np.float32)
        
        #if self.transforms:
        #    pass
            #spec, sr = self.transforms(data=(y, sr))['data']
            #spec = np.expand_dims(spec, axis=0) #np.transpose(spec, (2, 0, 1)).astype(np.float32)

            
        labels = np.zeros(len(BIRD_CODE), dtype="f")
        if self.train:
            if np.random.random() > 0.5:
                labels[BIRD_CODE[ebird_code]] = pred_prob
            else:
                labels[BIRD_CODE[ebird_code]] = 1
        #labels = BIRD_CODE[ebird_code]
        else:
            #ebird_code = ebird_code.split(' ')
            #print(ebird_code)
            #for eb in ebird_code:
            labels[BIRD_CODE[ebird_code]] = 1


        return {
            "waveform" : y, #torch.tensor(spec, dtype=torch.float),
            "targets" : labels, #torch.tensor(labels, dtype=torch.long)
        }

    def __get_labels__(self):
        return np.array(self.file_list)[:, 1]


Overwriting dataset.py


In [ ]:
%%writefile audio_albu.py

import audiomentations as A

augmenter = A.Compose([
    A.AddGaussianNoise(p=0.3),
    A.AddGaussianSNR(p=0.3),
    #A.AddBackgroundNoise("../input/train_audio/", p=1)
    #A.AddImpulseResponse(p=1),
    #A.AddShortNoises("../input/train_audio/", p=1)
    A.FrequencyMask(min_frequency_band=0.0,  max_frequency_band=0.2, p=0.2),
    A.TimeMask(min_band_part=0.0, max_band_part=0.2, p=0.2),
    A.PitchShift(min_semitones=-0.5, max_semitones=0.5, p=0.3),
    A.Shift(p=0.3),
    A.Normalize(p=0.3),
    A.ClippingDistortion(min_percentile_threshold=0, max_percentile_threshold=1, p=0.3),
    A.PolarityInversion(p=0.3),
    A.Gain(p=0.3)
])

Overwriting audio_albu.py


In [ ]:
%%writefile utils.py

import warnings
warnings.filterwarnings('ignore')

import torch
import numpy as np
from sklearn import metrics

def convert_onehot(x, max=54):
    return np.eye(max)[x]

def row_wise_f1_score_micro_numpy(y_true, y_pred, threshold=0.5, count=5):
    """ 
    @author shonenkov 
    
    y_true - 2d npy vector with gt
    y_pred - 2d npy vector with prediction
    threshold - for round labels
    count - number of preds (used sorting by confidence)
    """
    def meth_agn_v2(x, threshold):
        idx, = np.where(x > threshold)
        return idx[np.argsort(x[idx])[::-1]]

    F1 = []
    for preds, trues in zip(y_pred, y_true):
        TP, FN, FP = 0, 0, 0
        preds = meth_agn_v2(preds, threshold)[:count]
        trues = meth_agn_v2(trues, threshold)
        for true in trues:
            if true in preds:
                TP += 1
            else:
                FN += 1
        for pred in preds:
            if pred not in trues:
                FP += 1
        F1.append(2*TP / (2*TP + FN + FP))
    return np.mean(F1)

class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

class MetricMeter(object):

    def __init__(self, train=True):
        self.reset()
        self.train = train
    
    def reset(self):
        self.y_true = []
        self.y_pred = []
        self.y_true_ = []
        self.y_pred_ = []
        self.acc = 0
        self.f1 = 0
        self.map = 0
    
    def update(self, y_true, y_pred):
        self.y_true.extend(y_true.cpu().detach().numpy().tolist()) 
        self.y_pred.extend(torch.sigmoid(y_pred).cpu().detach().numpy().tolist())
        
    
    @property
    def avg(self):

        #self.y_true = np.concatenate(self.y_true, axis=0)
        #self.y_pred = np.concatenate(self.y_pred, axis=0)

        #print(self.y_true[0])
        #print(self.y_pred[0])

        #print(self.y_ture.shape, self.y_pred.shape)

        if self.train:
            self.acc = metrics.accuracy_score(np.argmax(self.y_true, axis=1), np.argmax(self.y_pred, axis=1))
            self.auc = 0#metrics.roc_auc_score(self.y_true, self.y_pred, average=None)
            self.f1 = 0#metrics.f1_score(np.argmax(self.y_true, axis=1), np.argmax(self.y_true, axis=1), average=None)
            self.map = 0#np.mean(metrics.average_precision_score(self.y_true, self.y_pred)
            self.row_f1 = 0
        else:
            self.acc = metrics.accuracy_score(np.argmax(self.y_true, axis=1), np.argmax(self.y_pred, axis=1))
            self.auc = 0 #np.mean(metrics.roc_auc_score(self.y_true, self.y_pred, average=None))
            self.f1 = metrics.f1_score(np.argmax(self.y_true, axis=1), np.argmax(self.y_pred, axis=1), average='macro')
            self.map = np.nan_to_num(
                    np.mean(metrics.average_precision_score(self.y_true, self.y_pred, average=None))
                ).mean()
            
            self.row_f1 = row_wise_f1_score_micro_numpy(np.array(self.y_true), np.array(self.y_pred))

        return {
            "acc" : self.acc,
            "f1" : self.f1,
            "auc" : self.auc,
            "map" : self.map,
            "row_f1" : self.row_f1
        }

Overwriting utils.py


In [ ]:
%%writefile losses.py

import torch
import torch.nn as nn
import torch.nn.functional as F

class PANNsLoss(nn.Module):
    def __init__(self):
        super().__init__()

        self.bce = nn.BCEWithLogitsLoss()
        self.cel = nn.CrossEntropyLoss()

    def forward(self, input, target):
        """
        input_ = input
        input_ = torch.where(
            torch.isnan(input_),
            torch.zeros_like(input_),
            input_
        )
        input_ = torch.where(
            torch.isinf(input_),
            torch.zeros_like(input_),
            input_
        )

        target = target.float()
        """

        #return self.bce(input_, target)
        return self.bce(input, target)

Overwriting losses.py


In [ ]:
%%writefile bird_codes.py

"""
BIRD_CODE = {
    'aldfly': 0, 'ameavo': 1, 'amebit': 2, 'amecro': 3, 'amegfi': 4,
    'amekes': 5, 'amepip': 6, 'amered': 7, 'amerob': 8, 'amewig': 9,
    'amewoo': 10, 'amtspa': 11, 'annhum': 12, 'astfly': 13, 'baisan': 14,
    'baleag': 15, 'balori': 16, 'banswa': 17, 'barswa': 18, 'bawwar': 19,
    'belkin1': 20, 'belspa2': 21, 'bewwre': 22, 'bkbcuc': 23, 'bkbmag1': 24,
    'bkbwar': 25, 'bkcchi': 26, 'bkchum': 27, 'bkhgro': 28, 'bkpwar': 29,
    'bktspa': 30, 'blkpho': 31, 'blugrb1': 32, 'blujay': 33, 'bnhcow': 34,
    'boboli': 35, 'bongul': 36, 'brdowl': 37, 'brebla': 38, 'brespa': 39,
    'brncre': 40, 'brnthr': 41, 'brthum': 42, 'brwhaw': 43, 'btbwar': 44,
    'btnwar': 45, 'btywar': 46, 'buffle': 47, 'buggna': 48, 'buhvir': 49,
    'bulori': 50, 'bushti': 51, 'buwtea': 52, 'buwwar': 53, 'nocall': 54
}

"""

BIRD_CODE = {
    'aldfly': 0, 'ameavo': 1, 'amebit': 2, 'amecro': 3, 'amegfi': 4,
    'amekes': 5, 'amepip': 6, 'amered': 7, 'amerob': 8, 'amewig': 9,
    'amewoo': 10, 'amtspa': 11, 'annhum': 12, 'astfly': 13, 'baisan': 14,
    'baleag': 15, 'balori': 16, 'banswa': 17, 'barswa': 18, 'bawwar': 19,
    'belkin1': 20, 'belspa2': 21, 'bewwre': 22, 'bkbcuc': 23, 'bkbmag1': 24,
    'bkbwar': 25, 'bkcchi': 26, 'bkchum': 27, 'bkhgro': 28, 'bkpwar': 29,
    'bktspa': 30, 'blkpho': 31, 'blugrb1': 32, 'blujay': 33, 'bnhcow': 34,
    'boboli': 35, 'bongul': 36, 'brdowl': 37, 'brebla': 38, 'brespa': 39,
    'brncre': 40, 'brnthr': 41, 'brthum': 42, 'brwhaw': 43, 'btbwar': 44,
    'btnwar': 45, 'btywar': 46, 'buffle': 47, 'buggna': 48, 'buhvir': 49,
    'bulori': 50, 'bushti': 51, 'buwtea': 52, 'buwwar': 53, 'cacwre': 54,
    'calgul': 55, 'calqua': 56, 'camwar': 57, 'cangoo': 58, 'canwar': 59,
    'canwre': 60, 'carwre': 61, 'casfin': 62, 'caster1': 63, 'casvir': 64,
    'cedwax': 65, 'chispa': 66, 'chiswi': 67, 'chswar': 68, 'chukar': 69,
    'clanut': 70, 'cliswa': 71, 'comgol': 72, 'comgra': 73, 'comloo': 74,
    'commer': 75, 'comnig': 76, 'comrav': 77, 'comred': 78, 'comter': 79,
    'comyel': 80, 'coohaw': 81, 'coshum': 82, 'cowscj1': 83, 'daejun': 84,
    'doccor': 85, 'dowwoo': 86, 'dusfly': 87, 'eargre': 88, 'easblu': 89,
    'easkin': 90, 'easmea': 91, 'easpho': 92, 'eastow': 93, 'eawpew': 94,
    'eucdov': 95, 'eursta': 96, 'evegro': 97, 'fiespa': 98, 'fiscro': 99,
    'foxspa': 100, 'gadwal': 101, 'gcrfin': 102, 'gnttow': 103, 'gnwtea': 104,
    'gockin': 105, 'gocspa': 106, 'goleag': 107, 'grbher3': 108, 'grcfly': 109,
    'greegr': 110, 'greroa': 111, 'greyel': 112, 'grhowl': 113, 'grnher': 114,
    'grtgra': 115, 'grycat': 116, 'gryfly': 117, 'haiwoo': 118, 'hamfly': 119,
    'hergul': 120, 'herthr': 121, 'hoomer': 122, 'hoowar': 123, 'horgre': 124,
    'horlar': 125, 'houfin': 126, 'houspa': 127, 'houwre': 128, 'indbun': 129,
    'juntit1': 130, 'killde': 131, 'labwoo': 132, 'larspa': 133, 'lazbun': 134,
    'leabit': 135, 'leafly': 136, 'leasan': 137, 'lecthr': 138, 'lesgol': 139,
    'lesnig': 140, 'lesyel': 141, 'lewwoo': 142, 'linspa': 143, 'lobcur': 144,
    'lobdow': 145, 'logshr': 146, 'lotduc': 147, 'louwat': 148, 'macwar': 149,
    'magwar': 150, 'mallar3': 151, 'marwre': 152, 'merlin': 153, 'moublu': 154,
    'mouchi': 155, 'moudov': 156, 'norcar': 157, 'norfli': 158, 'norhar2': 159,
    'normoc': 160, 'norpar': 161, 'norpin': 162, 'norsho': 163, 'norwat': 164,
    'nrwswa': 165, 'nutwoo': 166, 'olsfly': 167, 'orcwar': 168, 'osprey': 169,
    'ovenbi1': 170, 'palwar': 171, 'pasfly': 172, 'pecsan': 173, 'perfal': 174,
    'phaino': 175, 'pibgre': 176, 'pilwoo': 177, 'pingro': 178, 'pinjay': 179,
    'pinsis': 180, 'pinwar': 181, 'plsvir': 182, 'prawar': 183, 'purfin': 184,
    'pygnut': 185, 'rebmer': 186, 'rebnut': 187, 'rebsap': 188, 'rebwoo': 189,
    'redcro': 190, 'redhea': 191, 'reevir1': 192, 'renpha': 193, 'reshaw': 194,
    'rethaw': 195, 'rewbla': 196, 'ribgul': 197, 'rinduc': 198, 'robgro': 199,
    'rocpig': 200, 'rocwre': 201, 'rthhum': 202, 'ruckin': 203, 'rudduc': 204,
    'rufgro': 205, 'rufhum': 206, 'rusbla': 207, 'sagspa1': 208, 'sagthr': 209,
    'savspa': 210, 'saypho': 211, 'scatan': 212, 'scoori': 213, 'semplo': 214,
    'semsan': 215, 'sheowl': 216, 'shshaw': 217, 'snobun': 218, 'snogoo': 219,
    'solsan': 220, 'sonspa': 221, 'sora': 222, 'sposan': 223, 'spotow': 224,
    'stejay': 225, 'swahaw': 226, 'swaspa': 227, 'swathr': 228, 'treswa': 229,
    'truswa': 230, 'tuftit': 231, 'tunswa': 232, 'veery': 233, 'vesspa': 234,
    'vigswa': 235, 'warvir': 236, 'wesblu': 237, 'wesgre': 238, 'weskin': 239,
    'wesmea': 240, 'wessan': 241, 'westan': 242, 'wewpew': 243, 'whbnut': 244,
    'whcspa': 245, 'whfibi': 246, 'whtspa': 247, 'whtswi': 248, 'wilfly': 249,
    'wilsni1': 250, 'wiltur': 251, 'winwre3': 252, 'wlswar': 253, 'wooduc': 254,
    'wooscj2': 255, 'woothr': 256, 'y00475': 257, 'yebfly': 258, 'yebsap': 259,
    'yehbla': 260, 'yelwar': 261, 'yerwar': 262, 'yetvir': 263
}

INV_BIRD_CODE = {v: k for k, v in BIRD_CODE.items()}

Overwriting bird_codes.py


In [ ]:
!pip -q install git+https://github.com/ildoonet/pytorch-gradual-warmup-lr.git >> quit

In [ ]:
%%writefile schedulers.py

from torch.optim.lr_scheduler import CosineAnnealingLR
# Fix Warmup Bug
from warmup_scheduler import GradualWarmupScheduler  # https://github.com/ildoonet/pytorch-gradual-warmup-lr


class GradualWarmupSchedulerV2(GradualWarmupScheduler):
    def __init__(self, optimizer, multiplier, total_epoch, after_scheduler=None):
        super(GradualWarmupSchedulerV2, self).__init__(optimizer, multiplier, total_epoch, after_scheduler)
    def get_lr(self):
        if self.last_epoch > self.total_epoch:
            if self.after_scheduler:
                if not self.finished:
                    self.after_scheduler.base_lrs = [base_lr * self.multiplier for base_lr in self.base_lrs]
                    self.finished = True
                return self.after_scheduler.get_lr()
            return [base_lr * self.multiplier for base_lr in self.base_lrs]
        if self.multiplier == 1.0:
            return [base_lr * (float(self.last_epoch) / self.total_epoch) for base_lr in self.base_lrs]
        else:
            return [base_lr * ((self.multiplier - 1.) * self.last_epoch / self.total_epoch + 1.) for base_lr in self.base_lrs]

Overwriting schedulers.py


In [ ]:
%%writefile train.py

import warnings
warnings.filterwarnings('ignore')

import cv2
import audioread
import logging
import os
import random
import time
import warnings

import librosa
import librosa.display as display
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import soundfile as sf
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data

from contextlib import contextmanager
from IPython.display import Audio
from pathlib import Path
from typing import Optional, List

from catalyst.data.sampler import DistributedSampler, BalanceClassSampler
#from catalyst.dl import SupervisedRunner, State, CallbackOrder, Callback, CheckpointCallback
from fastprogress import progress_bar
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, average_precision_score

import classifiers
import losses
import dataset

from audio_albu import augmenter
from config import args
from utils import AverageMeter, MetricMeter
from schedulers import GradualWarmupSchedulerV2
from pytorch_utils import do_mixup, Mixup, move_data_to_device

mixup_augmenter = Mixup(mixup_alpha=1.)

def mixup_data(x, y, alpha=1.0, use_cuda=True):
    
    #if alpha > 0:
    #    lam = np.random.beta(alpha, alpha)
    #else:
    #    lam = 1
    
    batch_size = x.size()[0]

    if use_cuda:
        index0 = torch.randperm(batch_size).cuda()
        index1 = torch.randperm(batch_size).cuda()
        index2 = torch.randperm(batch_size).cuda()
        index3 = torch.randperm(batch_size).cuda()
        index4 = torch.randperm(batch_size).cuda()

    else:
        index = torch.randperam(bath_size)
    
    #mixed_x = lam * x + (1 - lam) * x[index, :]

    ind = random.choice([0,1,2,3,4])

    if ind == 0:
        mixed_x = x
        mixed_y = y
    elif ind == 1:
        mixed_x = torch.cat([x, x[index1, :]], dim=1)
        mixed_y = y + y[index1, :]
    elif ind == 2:
        mixed_x = torch.cat([x, x[index1, :], x[index2]], dim=1)
        mixed_y = y + y[index1, :] + y[index2, :]
    elif ind == 3:
        mixed_x = torch.cat([x, x[index1, :], x[index2], x[index3, :]], dim=1)
        mixed_y = y + y[index1, :] + y[index2, :] + y[index3, :]
    elif ind == 4:
        mixed_x = torch.cat([x, x[index1, :], x[index2], x[index3, :], x[index4, :]], dim=1)
        mixed_y = y + y[index1, :] + y[index2, :] + y[index3, :] + y[index4, :]
    
    #mixed_y = torch.clamp(mixed_y, min=0, max=1)

    """
    if np.random.random() > 0.5:
        mixed_x = lam * x + (1 - lam) * x[index, :]
        mixed_y = lam * y + (1 - lam) * y[index, :]
    else:
        mixed_x =  x + x[index, :]
        mixed_y =  y + y[index, :]
        
    """
    return mixed_x, mixed_y

def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)


def train_epoch(args, model, loader, criterion, optimizer, epoch):
    losses = AverageMeter()
    scores = MetricMeter(train=True)

    model.train()
    scaler = torch.cuda.amp.GradScaler()

    t = tqdm(loader, total=1000)
    for i, sample in enumerate(t):

        optimizer.zero_grad()

        input = sample['waveform'].to(args.device)
        target = sample['targets'].to(args.device)

        if args.new_mixup:
            input, target = mixup_data(input, target, args.alpha, True)
        
        ran = np.random.random()

        if args.mixup and ran > 0.5:
            mixup_lambda = mixup_augmenter.get_lambda(
                batch_size=len(input)
            )
            mixup_lambda = move_data_to_device(mixup_lambda, args.device)

            target = do_mixup(target, mixup_lambda)

        with torch.cuda.amp.autocast(enabled=False):
            if args.mixup and ran > 0.5:
                output = model(input, True, mixup_lambda)
            else:
                output = model(input, True)
            
            #print(target.sum(dim=1))
            #print(input.shape)

            loss = criterion(output, target)
        
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        #loss.backward()
        #optimizer.step()
        #scheduler.step()

        bs = input.size(0)
        scores.update(target, output)
        losses.update(loss.item(), bs)

        t.set_description(f"Train E:{epoch} - Loss:{losses.avg:0.4f}")

        if i == 1000:
            break

    t.close()

    return scores.avg, losses.avg

def valid_epoch(args, model, loader, criterion, epoch):
    losses = AverageMeter()
    scores = MetricMeter(train=False)

    model.eval()

    with torch.no_grad():
        t = tqdm(loader)
        for i, sample in enumerate(t):

            input = sample['waveform'].to(args.device)
            target = sample['targets'].to(args.device)

            output = model(input)
            loss = criterion(output, target)

            bs = input.size(0)
            scores.update(target, output)
            losses.update(loss.item(), bs)

            t.set_description(f"Valid E:{epoch} - Loss:{losses.avg:0.4f}")

    t.close()

    return scores.avg, losses.avg

def main(fold):

    # Setting seed
    seed = args.seed
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

    args.fold = fold
    args.save_path = os.path.join(args.output_dir, args.exp_name)
    os.makedirs(args.save_path, exist_ok=True)

    model = classifiers.__dict__[args.network](**args.model_config)
    if args.pretrained_path:
        weights = torch.load(args.pretrained_path, map_location=args.device)
        model.load_state_dict(weights["model"], strict=False)
    
    model = model.to(args.device)

    criterion = losses.__dict__[args.losses]()

    optimizer = torch.optim.Adam(model.parameters(), lr=args.init_lr)

    scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, args.epochs)
    scheduler_warmup = GradualWarmupSchedulerV2(optimizer, multiplier=10, total_epoch=1, after_scheduler=scheduler_cosine)
    
    train_df = pd.read_csv(args.train_csv)
    #valid_df = pd.read_csv(args.valid_csv)
    if args.DEBUG:
        train_df = train_df.sample(1000)
        #valid_df = valid_df.sample(1000)
    
    train_fold = train_df[train_df.kfold != fold]
    valid_fold = train_df[train_df.kfold.isin([fold])]

    #print(valid_fold.head())

    valid_fold = valid_fold[valid_fold.ebird_code == valid_fold.pred_code]

    #print(valid_fold.head())

    train_file_list = train_fold[['path', 'pred_code', 'pred_prob']].values.tolist()
    valid_file_list = valid_fold[['path', 'pred_code', 'pred_prob']].values.tolist()

    train_dataset = dataset.PANNsDataset(
        train_file_list,
        period=args.PERIOD,
        transforms=augmenter,#train_transforms(args.mel_param),
        train=True
    )
    valid_dataset = dataset.PANNsDataset(
        valid_file_list,
        period=args.PERIOD,
        transforms=False,#valid_transforms(args.mel_param)
        train=False
    )

    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=args.batch_size,
        #sampler = BalanceClassSampler(labels=train_dataset.__get_labels__(), mode="upsampling"),
        shuffle=True,
        drop_last=True,
        num_workers=args.num_workers
    )
    valid_loader = torch.utils.data.DataLoader(
        valid_dataset,
        batch_size=args.batch_size,
        shuffle=False,
        drop_last=False,
        num_workers=args.num_workers
    )

    print(".................... Training started .................")

    best_map = -999999
    best_f1  = -999999
    best_row_f1 = -99999

    if args.load_from:
        # "drive/My Drive/Cornell Birdcall Identification/weights/Cnn14_16k_5"
        weights = torch.load(os.path.join("drive/My Drive/Cornell Birdcall Identification/weights/Cnn14_16k_5", f"fold-{args.fold}.bin"))
        model.load_state_dict(weights["model"], strict=False)
        #optimizer.load_state_dict(weights["optimizer"])
        #scheduler_warmup.load_state_dict(weights["scheduler_warmup"])
        #args.start_epoch = weights["epoch"] + 1
        #best_map = 0.6212948058165978
        
        model = model.to(args.device)


    for epoch in range(args.start_epoch, args.epochs):

        #scheduler_warmup.step(epoch)

        train_avg, train_loss = train_epoch(
            args,
            model,
            train_loader,
            criterion,
            optimizer,
            epoch
        )

        valid_avg, valid_loss = valid_epoch(
            args,
            model,
            valid_loader,
            criterion,
            epoch
        )

        scheduler_warmup.step()    
        if epoch==2: scheduler_warmup.step() # bug workaround 

        content = f"""
                {time.ctime()} \n
                Fold:{args.fold}, Epoch:{epoch}, lr:{optimizer.param_groups[0]['lr']:.7}, \n
                Train Loss:{train_loss:0.4f} - ACC:{train_avg['acc']:0.4f} - F1:{train_avg['f1']:0.4f} - MAP:{train_avg['map']:0.4f}  \n
                Valid Loss:{valid_loss:0.4f} - ACC:{valid_avg['acc']:0.4f} - F1:{valid_avg['f1']:0.4f} - ROW_F1:{valid_avg['row_f1']:0.4f} - MAP:{valid_avg['map']:0.4f} \n\n
        """
        print(content)

        with open(f'{args.save_path}/log_{args.exp_name}.txt', 'a') as appender:
            appender.write(content + '\n')
        
        if valid_avg["map"] > best_map:
            print(f"######### >>>>>>> Model Improved from {best_map} -----> {valid_avg['map']}")
            checkpoint = { 
                'epoch': epoch,
                'model': model.state_dict(),
                'optimizer': optimizer.state_dict(),
                'scheduler_warmup': scheduler_warmup.state_dict()
            }

            torch.save(checkpoint, os.path.join(args.save_path, f"checkpoint-fold-{args.fold}.bin"))
            torch.save(model.state_dict(), os.path.join(args.save_path, f"fold-{args.fold}.bin"))

            best_map = valid_avg["map"]
        
        torch.save(model.state_dict(), os.path.join(args.save_path, f"last-fold-{args.fold}.bin"))

if __name__ == "__main__":
    
    for fold in range(5):
        print("#"*20)
        print(f"####### FOLD : {fold}")
        if fold == 0:
            main(fold)
        

Overwriting train.py


In [ ]:
!wget https://zenodo.org/record/3987831/files/Cnn14_DecisionLevelAtt_mAP%3D0.425.pth?download=1

--2020-09-12 08:29:03--  https://zenodo.org/record/3987831/files/Cnn14_DecisionLevelAtt_mAP%3D0.425.pth?download=1
Resolving zenodo.org (zenodo.org)... 188.184.117.155
Connecting to zenodo.org (zenodo.org)|188.184.117.155|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 331757477 (316M) [application/octet-stream]
Saving to: ‘Cnn14_DecisionLevelAtt_mAP=0.425.pth?download=1.1’

Cnn14_DecisionLevel 100%[===================>] 316.39M  6.00MB/s    in 45s     

2020-09-12 08:29:50 (6.99 MB/s) - ‘Cnn14_DecisionLevelAtt_mAP=0.425.pth?download=1.1’ saved [331757477/331757477]



In [ ]:
%%writefile config.py

class args:

    DEBUG = False

    mixup = True

    new_mixup = False

    alpha = 1

    exp_name = "stage1_Cnn14_DecisionLevelAtt_1sec_aug"
    output_dir = "drive/My Drive/Cornell Birdcall Identification/weights"
    train_csv = "drive/My Drive/Cornell Birdcall Identification/input/stage1_sudo.csv"
    valid_csv = "drive/My Drive/Cornell Birdcall Identification/input/valid_stage1_df3.csv"
    pretrained_path = "Cnn14_DecisionLevelAtt_mAP=0.425.pth?download=1" #False #"Cnn14_16k_mAP=0.438.pth?download=1"

    network = "Cnn14_DecisionLevelAtt" #"Cnn14_16k" #"BirdClassifier"
    encoder = "resnest50d"

    losses = "PANNsLoss"

    mel_param = {
        #"hop_lenght" : 345 * 2,
        "fmin" : 20,
        "fmax" : 16000 // 2,
        "n_mels" : 128,
        "n_fft" : 128 * 20
    }

    model_config = {
        #"encoder" : "resnest50d",
        "sample_rate": 16000,
        "window_size": 512,
        "hop_size": 160,
        "mel_bins": 64,
        "fmin": 50,
        "fmax": 8000,
        "classes_num": 264 #209 #54 #264
    }

    PERIOD = 1
    
    device = "cuda"
    seed = 42
    epochs = 50
    batch_size = 32 * 4
    num_workers = 4
    start_epoch = 0

    warmup_epo = 2
    cosine_epo = 8
    init_lr = 3e-5

    load_from = False


Overwriting config.py


In [ ]:
!python3 train.py

####################
####### FOLD : 0
.................... Training started .................
Train E:0 - Loss:0.7652: 100% 1000/1000 [05:37<00:00,  2.97it/s]
Valid E:0 - Loss:0.6953: 100% 616/616 [01:15<00:00,  8.15it/s]

                Sat Sep 12 08:37:33 2020 

                Fold:0, Epoch:0, lr:0.0003, 

                Train Loss:0.7652 - ACC:0.0044 - F1:0.0000 - MAP:0.0000  

                Valid Loss:0.6953 - ACC:0.0049 - F1:0.0004 - ROW_F1:0.0059 - MAP:0.0046 


        
######### >>>>>>> Model Improved from -999999 -----> 0.004605226564525975
Train E:1 - Loss:0.6932: 100% 1000/1000 [05:38<00:00,  2.96it/s]
Valid E:1 - Loss:0.6931: 100% 616/616 [00:46<00:00, 13.25it/s]

                Sat Sep 12 08:44:24 2020 

                Fold:0, Epoch:1, lr:0.0003, 

                Train Loss:0.6932 - ACC:0.0041 - F1:0.0000 - MAP:0.0000  

                Valid Loss:0.6931 - ACC:0.0053 - F1:0.0006 - ROW_F1:0.0063 - MAP:0.0048 


        
######### >>>>>>> Model Improved from 0.004605

In [ ]:
!python3 train.py

####################
####### FOLD : 0
.................... Training started .................
Train E:0 - Loss:0.0133: 100% 1000/1000 [04:54<00:00,  3.40it/s]
Valid E:0 - Loss:0.0043: 100% 833/833 [01:43<00:00,  8.04it/s]

                Sat Sep 12 05:43:56 2020 

                Fold:0, Epoch:0, lr:0.0003, 

                Train Loss:0.0133 - ACC:0.3695 - F1:0.0000 - MAP:0.0000  

                Valid Loss:0.0043 - ACC:0.8647 - F1:0.8357 - ROW_F1:0.7595 - MAP:0.8876 


        
######### >>>>>>> Model Improved from -999999 -----> 0.8876025344539631
Train E:1 - Loss:0.0116: 100% 1000/1000 [04:56<00:00,  3.37it/s]
Valid E:1 - Loss:0.0040: 100% 833/833 [01:42<00:00,  8.14it/s]

                Sat Sep 12 05:51:08 2020 

                Fold:0, Epoch:1, lr:0.0003, 

                Train Loss:0.0116 - ACC:0.4465 - F1:0.0000 - MAP:0.0000  

                Valid Loss:0.0040 - ACC:0.8519 - F1:0.8167 - ROW_F1:0.7587 - MAP:0.8932 


        
######### >>>>>>> Model Improved from 0.88760253